# Training 
# - Loading images 
# - Build a simple CNN model and perform training
# - Deployment of model (waiting for AWS work account to activate first)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
# path variables
PROJ_PATH = os.path.abspath(os.curdir)
PATH_TRAIN_DIR = os.path.join(PROJ_PATH, 'dataset\\train')
print(PROJ_PATH)

d:\Work\IMDA\capstone project\projects\CardPrediction


In [3]:
# load data from training folder, split 80% as training, 20% as validation
BATCH_SIZE = 64
IMG_SIZE = (160, 160)
EPOCHS = 10

train = image_dataset_from_directory(
    PATH_TRAIN_DIR, 
    labels='inferred', 
    shuffle=True, 
    batch_size=BATCH_SIZE, 
    image_size=IMG_SIZE,
    validation_split=0.2,
    subset='training',
    seed=23
    )
    
validation = image_dataset_from_directory(
    PATH_TRAIN_DIR, 
    labels='inferred', 
    shuffle=True, 
    batch_size=BATCH_SIZE, 
    image_size=IMG_SIZE,
    validation_split=0.2,
    subset='validation',
    seed=23
    )

num_of_classes = len(train.class_names)

# apply bufferred prefetching to improve training performance
AUTOTUNE = tf.data.AUTOTUNE
train = train.prefetch(buffer_size=AUTOTUNE)
validation = validation.prefetch(buffer_size=AUTOTUNE)

Found 1260 files belonging to 12 classes.
Using 1008 files for training.
Found 1260 files belonging to 12 classes.
Using 252 files for validation.


In [11]:
# create model
model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(160, 160, 3)),
  layers.Conv2D(8, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dropout(0.2),
  layers.Dense(32, activation='relu'),
  layers.Dropout(0.2),
  layers.Dense(num_of_classes)
])

# compile model and train
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(
  train,
  validation_data=validation,
  epochs=EPOCHS
)

Epoch 1/10
16/16 [==============================] - 6s 321ms/step - loss: 2.2974 - accuracy: 0.2133 - val_loss: 0.8433 - val_accuracy: 1.0000
Epoch 2/10
16/16 [==============================] - 6s 318ms/step - loss: 0.7631 - accuracy: 0.8274 - val_loss: 0.0281 - val_accuracy: 1.0000
Epoch 3/10
16/16 [==============================] - 6s 321ms/step - loss: 0.1670 - accuracy: 0.9459 - val_loss: 0.0044 - val_accuracy: 1.0000
Epoch 4/10
16/16 [==============================] - 6s 324ms/step - loss: 0.0799 - accuracy: 0.9709 - val_loss: 0.0064 - val_accuracy: 1.0000
Epoch 5/10
16/16 [==============================] - 6s 318ms/step - loss: 0.0487 - accuracy: 0.9876 - val_loss: 4.4921e-04 - val_accuracy: 1.0000
Epoch 6/10
16/16 [==============================] - 6s 326ms/step - loss: 0.0311 - accuracy: 0.9937 - val_loss: 1.5411e-04 - val_accuracy: 1.0000
Epoch 7/10
16/16 [==============================] - 6s 322ms/step - loss: 0.0291 - accuracy: 0.9935 - val_loss: 3.0394e-04 - val_accuracy: 1

In [16]:
score = model.evaluate(validation)
print('[validation] loss:', score[0], ' accuracy:', score[1])

4/4 [==============================] - 1s 76ms/step - loss: 1.4628e-04 - accuracy: 1.0000
[validation] loss: 0.00014628292410634458  accuracy: 1.0


## Visualization

In [15]:
PATH_TRAIN_DIR = PROJ_PATH + '\\dataset\\train\\'
PATH_IMG_DIR = PROJ_PATH + '\\dataset\\images\\'

labels = []
print('Loading training dataset...')
for index, directory in enumerate(os.listdir(PATH_TRAIN_DIR)):
    labels.append(directory)
print('Completed!')

def prediction(card_path):
    img = tf.keras.preprocessing.image.load_img(
        card_path, target_size=(160, 160)
    )
    #plt.figure()
    #plt.imshow(img)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    prediction = model.predict(img_array)
    print('predicted : ' + labels[np.argmax(prediction)])

#prediction(PATH_TRAIN_DIR+'CHUN-LI 001/CHUN-LI 001_0_2.png')

print('Performing prediction...')
for index, filename in enumerate(os.listdir(PATH_IMG_DIR)):
    print("*******************")
    print('Input image ' + filename)
    prediction(card_path = PATH_IMG_DIR + filename)
print('Completed!')

Loading training dataset...
Completed!
Performing prediction...
*******************
Input image CHUN-LI 001.png
predicted : CHUN-LI 001
*******************
Input image CHUN-LI 002.png
predicted : CHUN-LI 002
*******************
Input image CHUN-LI 003.png
predicted : CHUN-LI 003
*******************
Input image CHUN-LI 004.png
predicted : CHUN-LI 004
*******************
Input image CHUN-LI 005.png
predicted : CHUN-LI 005
*******************
Input image CHUN-LI 006.png
predicted : CHUN-LI 006
*******************
Input image CHUN-LI 007.png
predicted : CHUN-LI 007
*******************
Input image CHUN-LI 008.png
predicted : CHUN-LI 008
*******************
Input image CHUN-LI 009.png
predicted : CHUN-LI 009
*******************
Input image CHUN-LI 010.png
predicted : CHUN-LI 010
*******************
Input image CHUN-LI 011.png
predicted : CHUN-LI 011
*******************
Input image CHUN-LI 012.png
predicted : CHUN-LI 012
*******************
Input image CHUN-LI 013.png
predicted : CHUN-LI 010
